%%time

import datetime as dt
import os

import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

from common.data_frame_functions.functions import Functions

functions: Functions = Functions()

In [2]:
%%time

# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture",	"trippinthroughtime",
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"fatsquirrelhate",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks"
]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 15.6 ms


In [4]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]


CPU times: total: 844 ms
Wall time: 2.22 s


In [5]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

'==== CURATED DATA ===='

(36636, 14)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13bya6k,bathandbodyworks,PetiteTXgirl,Late March-April empties,there are many different types of skin care pr...,71169906f34149ab001ecc885898d9a9,/r/bathandbodyworks/comments/13bya6k/late_marc...,https://i.redd.it/7tivxy31moya1.jpg,13bya6k.jpg,data/image/13bya6k.jpg,CandleDiffusion,True,True,False,[]
13byvzd,bathandbodyworks,iluvsunni,Some semi-oldies,there are two bottles of perfume sitting on a ...,031430dae686b90afe2f602d81d4df2e,/r/bathandbodyworks/comments/13byvzd/some_semi...,https://i.redd.it/c4qskam2qoya1.jpg,13byvzd.jpg,data/image/13byvzd.jpg,CandleDiffusion,True,True,True,[]
13bzh6l,bathandbodyworks,huskyyyy21,my summer lotion line up 🪼🌞🌴🩵,a close up of a table with various types of bo...,35d94f4e9675c8d8d4964b2c03d73b4c,/r/bathandbodyworks/comments/13bzh6l/my_summer...,https://i.redd.it/eezk3i7ytoya1.jpg,13bzh6l.jpg,data/image/13bzh6l.jpg,CandleDiffusion,True,True,True,[]


CPU times: total: 766 ms
Wall time: 906 ms


In [6]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

'==== UPDATED EXTANT DATA ===='

(41505, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,131jvcx.jpg,data/image/131jvcx.jpg,PrettyGirlDiffusion,True,True,False,[]


CPU times: total: 859 ms
Wall time: 4.09 s


In [7]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

CPU times: total: 109 ms
Wall time: 190 ms


In [8]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 17/17 [00:00<00:00, 2389.76it/s]


In [9]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				for submission in tqdm(subreddit_stream, total=len(subreddit_stream),
									   desc=f"Posts - {sub} - {time_filter}"):
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

day: 100%|██████████| 17/17 [00:24<00:00,  1.47s/it]


In [10]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

Reading Temp Dir For Subs...: 100%|██████████| 17/17 [00:10<00:00,  1.67it/s]


'==== TEMP DATA ===='

(2245, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
133garc,133garc,SFWRedheads,festivalfashionista,In my fave uniform,,,/r/SFWRedheads/comments/133garc/in_my_fave_uni...,https://i.imgur.com/O2CHhag.gifv,133garc.jpg,,,False,False,False,[]
13571tn,13571tn,SFWRedheads,Anikaprincess,Sexy or no?,,,/r/SFWRedheads/comments/13571tn/sexy_or_no/,https://i.imgur.com/1wUudk8.gifv,13571tn.jpg,,,False,False,False,[]
135ezva,135ezva,SFWRedheads,Historical_Ratio_294,Not usually a red lipstick kinda girl...,,,/r/SFWRedheads/comments/135ezva/not_usually_a_...,https://i.redd.it/4nb3ruke5fxa1.png,135ezva.jpg,,,False,False,False,[]
135g58p,135g58p,SFWRedheads,Scarlett_somebody,Sunshine ☀️,,,/r/SFWRedheads/comments/135g58p/sunshine/,https://www.reddit.com/gallery/135g58p,135g58p.jpg,,,False,False,False,[]
1360008,1360008,SFWRedheads,MalloyPanda,Sweater for Chilly Spring Weather,,,/r/SFWRedheads/comments/1360008/sweater_for_ch...,https://v.redd.it/md4388wcwixa1,1360008.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d97vz,13d97vz,bathandbodyworks,FreshPrincess90,I'm so obsessed with these two candles! What c...,,,/r/bathandbodyworks/comments/13d97vz/im_so_obs...,https://www.reddit.com/gallery/13d97vz,13d97vz.jpg,,,False,False,False,[]
13dagbr,13dagbr,bathandbodyworks,Dove04,Does anyone know how I could get rid of this r...,,,/r/bathandbodyworks/comments/13dagbr/does_anyo...,https://i.redd.it/p0j6atz6rxya1.jpg,13dagbr.jpg,,,False,False,False,[]
13daik7,13daik7,bathandbodyworks,Emergency_Room_777,Issues with hyaluronic acid body creams....Try...,,,/r/bathandbodyworks/comments/13daik7/issues_wi...,https://www.reddit.com/r/bathandbodyworks/comm...,13daik7.jpg,,,False,False,False,[]


CPU times: total: 21.8 s
Wall time: 23.3 s


In [11]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]
filtered.dropna(axis=1, how='all')
filtered.reset_index(drop=True, inplace=True)
filtered.set_index('id', inplace=True, drop=False)
display("==== FILTERED DATA ====")
display(filtered.shape)
display(filtered)

'==== FILTERED DATA ===='

(406, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,,False,False,False,[]
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,,,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,,,False,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,,,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,,,False,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,,,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,,,False,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,,,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,,,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,,,False,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,,,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,,,False,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,,,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,,,False,False,False,[]


CPU times: total: 109 ms
Wall time: 111 ms


In [12]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 406/406 [00:00<00:00, 16235.14it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Model =='

(406, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,,,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,,RedHeadDiffusion,False,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,,,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,,RedHeadDiffusion,False,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,,,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,,RedHeadDiffusion,False,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,,,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,,,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,,CandleDiffusion,False,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,,,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,,CandleDiffusion,False,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,,,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,,CandleDiffusion,False,False,False,[]


CPU times: total: 109 ms
Wall time: 100 ms


In [13]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 406/406 [05:05<00:00,  1.33it/s]
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Path =='

(406, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,,,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,False,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,,,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,False,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,,,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,False,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,,,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,,,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,False,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,,,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,False,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,,,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,False,False,False,[]


CPU times: total: 2min 59s
Wall time: 7min 4s


In [14]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 406/406 [00:00<00:00, 2103.63it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Exists =='

(406, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,,,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,True,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,,,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,True,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,,,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,True,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,,,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,,,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,,,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,,,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 250 ms
Wall time: 344 ms


In [15]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 406/406 [00:03<00:00, 128.32it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Hash =='

(406, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,,0c2da695483dd7562ec9ef13f791b934,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,True,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,,4c3fe8876deccd1f3e9f19ba3d8f5cfc,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,True,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,,58382c40ac34f2fa5b23666e9c939948,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,True,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,,bbc65198cbe68d8efdc6f45187939f68,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 1.25 s
Wall time: 3.36 s


In [16]:
%%time

caption_0 = BlipCaption("cuda:0")
caption_1 = BlipCaption("cuda:1")

CPU times: total: 2min 11s
Wall time: 2min 52s


In [17]:
%%time

filtered['caption'] = filtered.progress_apply(lambda x: functions.apply_caption(x, [caption_0, caption_1]), axis=1)

display("== Filtered Data With Caption ==")
display(filtered.shape)
display(filtered)

Progress:   0%|          | 0/406 [00:00<?, ?it/s]D:\code\repos\simple-collection\venv\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Progress: 100%|██████████| 406/406 [15:42<00:00,  2.32s/it]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Caption =='

(406, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13c3bc5,13c3bc5,SFWRedheads,Redhotwife88,Redhead Wife in my Little Black Dress! [F],,,/r/SFWRedheads/comments/13c3bc5/redhead_wife_i...,https://i.redgifs.com/i/admiredoutstandingpape...,13c3bc5.jpg,,RedHeadDiffusion,False,False,False,[]
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,there is a woman with glasses and a black top ...,0c2da695483dd7562ec9ef13f791b934,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,True,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,there is a woman with red hair and a brown top,4c3fe8876deccd1f3e9f19ba3d8f5cfc,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,True,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,arafed woman sitting in a car with her hand on...,58382c40ac34f2fa5b23666e9c939948,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,True,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,arafed woman with red hair and blue eyes weari...,bbc65198cbe68d8efdc6f45187939f68,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 14min 58s
Wall time: 15min 42s


In [18]:
%%time

filtered_more = filtered.loc[(filtered['caption'] != "") & (filtered['caption'].notnull()) & (filtered['exists'] == True)]

display("== Filtered On Missing Caption ==")
display(filtered_more.shape)
display(filtered_more)

'== Filtered On Missing Caption =='

(395, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,there is a woman with glasses and a black top ...,0c2da695483dd7562ec9ef13f791b934,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,True,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,there is a woman with red hair and a brown top,4c3fe8876deccd1f3e9f19ba3d8f5cfc,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,True,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,arafed woman sitting in a car with her hand on...,58382c40ac34f2fa5b23666e9c939948,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,True,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,arafed woman with red hair and blue eyes weari...,bbc65198cbe68d8efdc6f45187939f68,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,True,False,False,[]
13ck3w5,13ck3w5,SFWRedheads,kisstheredfox,Good morning from the studio,there is a woman with long red hair posing for...,46882166f463eccc64e63b698c975234,/r/SFWRedheads/comments/13ck3w5/good_morning_f...,https://i.redd.it/jso53o4zsrya1.jpg,13ck3w5.jpg,data/image/13ck3w5.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 78.1 ms
Wall time: 143 ms


In [19]:
%%time

dropped = filtered_more.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)

display("== Dropped And Reset Data ==")
display(dropped.shape)
display(dropped)

'== Dropped And Reset Data =='

(395, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,there is a woman with glasses and a black top ...,0c2da695483dd7562ec9ef13f791b934,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,True,False,False,[]
1,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,there is a woman with red hair and a brown top,4c3fe8876deccd1f3e9f19ba3d8f5cfc,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,True,False,False,[]
2,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,arafed woman sitting in a car with her hand on...,58382c40ac34f2fa5b23666e9c939948,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,True,False,False,[]
3,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,arafed woman with red hair and blue eyes weari...,bbc65198cbe68d8efdc6f45187939f68,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,True,False,False,[]
4,13ck3w5,SFWRedheads,kisstheredfox,Good morning from the studio,there is a woman with long red hair posing for...,46882166f463eccc64e63b698c975234,/r/SFWRedheads/comments/13ck3w5/good_morning_f...,https://i.redd.it/jso53o4zsrya1.jpg,13ck3w5.jpg,data/image/13ck3w5.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
391,13d661b,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
392,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]
393,13dagbr,bathandbodyworks,Dove04,Does anyone know how I could get rid of this r...,someone is holding a white device with a hole ...,41732e4109a99c99c91c2181867d18e3,/r/bathandbodyworks/comments/13dagbr/does_anyo...,https://i.redd.it/p0j6atz6rxya1.jpg,13dagbr.jpg,data/image/13dagbr.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 125 ms
Wall time: 128 ms


In [20]:
%%time

current = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system)

display(current.shape)
display(current)

(36636, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36631,13bya6k,bathandbodyworks,PetiteTXgirl,Late March-April empties,there are many different types of skin care pr...,71169906f34149ab001ecc885898d9a9,/r/bathandbodyworks/comments/13bya6k/late_marc...,https://i.redd.it/7tivxy31moya1.jpg,13bya6k.jpg,data/image/13bya6k.jpg,CandleDiffusion,True,True,False,[]
36632,13byvzd,bathandbodyworks,iluvsunni,Some semi-oldies,there are two bottles of perfume sitting on a ...,031430dae686b90afe2f602d81d4df2e,/r/bathandbodyworks/comments/13byvzd/some_semi...,https://i.redd.it/c4qskam2qoya1.jpg,13byvzd.jpg,data/image/13byvzd.jpg,CandleDiffusion,True,True,True,[]
36633,13bzh6l,bathandbodyworks,huskyyyy21,my summer lotion line up 🪼🌞🌴🩵,a close up of a table with various types of bo...,35d94f4e9675c8d8d4964b2c03d73b4c,/r/bathandbodyworks/comments/13bzh6l/my_summer...,https://i.redd.it/eezk3i7ytoya1.jpg,13bzh6l.jpg,data/image/13bzh6l.jpg,CandleDiffusion,True,True,True,[]
36634,13c2k1d,bathandbodyworks,xeloux,"Rainy day here, decided to burn for cozy. What...",there is a candle that is sitting on a table,09388a91d70ee4e5f03211f7c457bfc9,/r/bathandbodyworks/comments/13c2k1d/rainy_day...,https://i.redd.it/w8c4gtb9dpya1.jpg,13c2k1d.jpg,data/image/13c2k1d.jpg,CandleDiffusion,True,True,True,[]


CPU times: total: 1.58 s
Wall time: 2.33 s


In [21]:
%%time

temp_current = current
temp_current.set_index('id', inplace=True, drop=False)

temp_new = dropped
temp_new.set_index('id', inplace=True, drop=False)

temp_new.loc[~temp_new.index.isin(temp_current.index)]

CPU times: total: 15.6 ms
Wall time: 21 ms


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13cbhj1,13cbhj1,SFWRedheads,itsSavannahWilde,Red hair & pink frames,there is a woman with glasses and a black top ...,0c2da695483dd7562ec9ef13f791b934,/r/SFWRedheads/comments/13cbhj1/red_hair_pink_...,https://i.redd.it/bnqlx1ku3rya1.jpg,13cbhj1.jpg,data/image/13cbhj1.jpg,RedHeadDiffusion,True,False,False,[]
13cf7m7,13cf7m7,SFWRedheads,ZealousidealEnd947,40 pounds down! Feeling ✨confident✨,there is a woman with red hair and a brown top,4c3fe8876deccd1f3e9f19ba3d8f5cfc,/r/SFWRedheads/comments/13cf7m7/40_pounds_down...,https://i.redd.it/vigduj3bxrya1.jpg,13cf7m7.jpg,data/image/13cf7m7.jpg,RedHeadDiffusion,True,False,False,[]
13cffyg,13cffyg,SFWRedheads,MortgageFormer1946,Tired 😴,arafed woman sitting in a car with her hand on...,58382c40ac34f2fa5b23666e9c939948,/r/SFWRedheads/comments/13cffyg/tired/,https://i.redd.it/vbtgztbdzrya1.jpg,13cffyg.jpg,data/image/13cffyg.jpg,RedHeadDiffusion,True,False,False,[]
13cfi2n,13cfi2n,SFWRedheads,istayyawayyyX,redheads-for those brave enough to play with f...,arafed woman with red hair and blue eyes weari...,bbc65198cbe68d8efdc6f45187939f68,/r/SFWRedheads/comments/13cfi2n/redheadsfor_th...,https://i.redd.it/rx899dkwzrya1.jpg,13cfi2n.jpg,data/image/13cfi2n.jpg,RedHeadDiffusion,True,False,False,[]
13ck3w5,13ck3w5,SFWRedheads,kisstheredfox,Good morning from the studio,there is a woman with long red hair posing for...,46882166f463eccc64e63b698c975234,/r/SFWRedheads/comments/13ck3w5/good_morning_f...,https://i.redd.it/jso53o4zsrya1.jpg,13ck3w5.jpg,data/image/13ck3w5.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]


In [22]:
%%time

concat = pd.concat([current, dropped])

display("== Concatenated Data ==")
display(concat.shape)
display(concat)

'== Concatenated Data =='

(37031, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100rn7k,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13d2hsf,13d2hsf,bathandbodyworks,alesieoksap,Ice Cream Shop Collection?,someone holding a cup of ice cream in their hand,c6788074138175c661fc652bd7630e33,/r/bathandbodyworks/comments/13d2hsf/ice_cream...,https://i.redd.it/uhwamoq99wya1.jpg,13d2hsf.jpg,data/image/13d2hsf.jpg,CandleDiffusion,True,False,False,[]
13d661b,13d661b,bathandbodyworks,xeloux,Facebook market finds!,three candles are sitting on a blanket on a bed,e868289445c195815dd67dd0df1a65cb,/r/bathandbodyworks/comments/13d661b/facebook_...,https://i.redd.it/skd10hefxwya1.jpg,13d661b.jpg,data/image/13d661b.jpg,CandleDiffusion,True,False,False,[]
13d8s0i,13d8s0i,bathandbodyworks,Alternative-Tea-9355,Black Cherry Merlot dupe,someone holding a tube of body cream in a store,965bc5a58a597192700234729758101b,/r/bathandbodyworks/comments/13d8s0i/black_che...,https://i.redd.it/xt7qew9qexya1.jpg,13d8s0i.jpg,data/image/13d8s0i.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 93.8 ms
Wall time: 187 ms


In [ ]:
%%time

file_list_ = file_system.ls("data/image")
concat['path'] = concat.progress_apply(lambda x: functions.fix_path(x, file_list), axis=1)

display("== Concatenated Data With Path ==")
display(concat.shape)
display(concat)

In [ ]:
%%time

back_up_name = f"data/parquet/back_{dt.datetime.timestamp(dt.datetime.now())}.parquet"
display(f"== Writing Back-Up {back_up_name} ==")
current.to_parquet(back_up_name, schema=schema, filesystem=file_system)

In [ ]:
%%time

concat.to_parquet("data/parquet/back.parquet", schema=schema, filesystem=file_system)
new = pd.read_parquet("data/parquet/back.parquet", engine='pyarrow', schema=schema, filesystem=file_system)

display("== New Data ==")
display(new.shape)
display(new)

In [ ]:
!jupyter notebook stop